### Ira pegar o sumario dos jogos e salvar os links para outros mercados e fontes de dados

só iremos trabalhar com dados diretamente relacionados por enquanto, mas iremos coletar todos, então, dados não relacionados como resultados das partidas da rodada passada serao coletados mas não limpos, assim como dados dos jogadores, juizes e estadios

In [1]:
import os
import pandas as pd
import requests
import bs4
import time
import numpy as np

In [2]:
def createDirectory(day_event_name=None, event=None):
    '''
        cria os diretorios para o dia do evento e tambem para cada evento
    '''
    try:
        if day_event_name != None:
            global day_event
            day_event = str(day_event_name.split('with')[0][:-1])
            
            global directory_name
            directory_name = './data_games' + '/' + day_event + '/'
            os.mkdir(directory_name)
        elif event != None:
            global dir_to_event
            back_team = df.loc[event, 'back_team']
            home_team = df.loc[event, 'home_team']
            global event_name
            event_name = home_team + '_v_' + back_team
            dir_to_event =  directory_name + event_name + '/'
            os.mkdir(dir_to_event)
        return True
    except Exception as error:
        if 'FileExistsError' in str(type(error)):
            pass
        else:
            print(type(error), error)

In [3]:
def saveHtml(event, step):
    '''
        recebe um evento da iteracao do loop principal e salva os seus html
    
    '''
    try:
        global back_team, home_team
        back_team = df.loc[event, 'back_team']
        home_team = df.loc[event, 'home_team']
        
        if step == 1:
            file_name = 'HTML_STATS_' + home_team + ' v ' + back_team + '.txt'
        elif step == 2:
            file_name = 'HTML_SUMMARY_' + home_team + ' v ' + back_team + '.txt'
        
        with open(dir_to_event + file_name, 'w') as filee:
            filee.write(str(req.content))
        return True
    except Exception as error:
        print(type(error), error)
        return False

In [4]:
def getDataFont():
    '''
        Ira pegar os links do menu principal com quatro fontes de dados:
        * Ficha de Jogo
        * Estatisticas
        * Previsoes
        * Tips
        
        Ira retornar apenas o link para o sumario do jogo
    '''
    menu_list = htmlPage.find('ul',{'class':'tabs boxed-tabs stats-tabs'})
    
    data_fonts = {'summary': None, 'stats': None, 'prediction': None, 'tips': None}
    for menu in menu_list.find_all('li'):
        menu_name = menu.text.replace('\n', '')
        if menu_name == 'Ficha de jogo':
            data_fonts['summary'] = menu.find('a').get('href')
        elif menu_name == 'Estatísticas':
            data_fonts['stats'] = menu.find('a').get('href')
        elif menu_name == 'Previsão':
            data_fonts['prediction'] = menu.find('a').get('href')
        elif menu_name == 'Tips':
            data_fonts['tips'] = menu.find('a').get('href')

    '''salva os dados em um arquivo csv'''
    saveDataFonts(data_fonts)
        
    return data_fonts['summary']

def saveDataFonts(data_fonts):
    '''
        Recebe um dicionario com os links com as fontes de dados e salva em csv
    '''
    df_data_fonts = pd.DataFrame(columns=list(data_fonts.keys()))
    df_data_fonts = df_data_fonts.append({'summary': data_fonts['summary'], 'stats': data_fonts['stats'], 'prediction': data_fonts['prediction'], 'tips': data_fonts['tips']}, ignore_index=True)
    
    file_name = 'DATA_FONTS_' + home_team + ' v ' + back_team + '.csv'
    df_data_fonts.to_csv(dir_to_event + file_name)
    return True

In [5]:
# DADOS DO ESTADIO
def getEstadioData():
    '''
        Pega os dados de estadio
    '''
    try:
        table_estadio = htmlPage.find('table',{'class':'match-stats-live-bg venue'})

        # a secao estadio esta dividido em duas colunas, em uma ha imagem e link para o estadio e noutra os dados
        estadio_link = table_estadio.find_all('td',{'width':'50%'})[0].find('a').get('href')
        estadio_img = table_estadio.find_all('td',{'width':'50%'})[0].find('img').get('src')

        lines = table_estadio.find_all('td',{'width':'50%'})[1].find_all('tr')

        try:
            estadio_name = lines[0].find_all('td')[1].text
        except:
            estadio_name = np.nan
        
        try:
            estadio_city = lines[1].find_all('td')[1].text
        except:
            estadio_city = np.nan
            
        try:    
            estadio_assistance = lines[2].find_all('td')[1].text
        except:
            estadio_assistance = np.nan
        
        try:
            estadio_capacity = lines[3].find_all('td')[1].text
        except:
            estadio_capacity = np.nan

        estadio_data = {'estadio_link': estadio_link, 'estadio_img': estadio_img, 'estadio_name': estadio_name,
                       'estadio_city': estadio_city, 'estadio_assistance': estadio_assistance, 'estadio_capacity': estadio_capacity}

        return estadio_data
    except Exception as error:
        if 'AttributeError' in str(type(error)):
            estadio_data = {'estadio_link': np.nan, 'estadio_img': np.nan, 'estadio_name': np.nan,
                       'estadio_city': np.nan, 'estadio_assistance': np.nan, 'estadio_capacity': np.nan}
        else:
            print('\nERROR [getEstadioData()]\n')
            print(type(error), error)
    
    return estadio_data

In [6]:
# PEGA OS JUIZES DO JOGO
def getJuizes():
    '''
        Pega os Juizes/Arbitros do jogo
        Dados nao estao limpos / not cleaned data
    '''
    table_linups = htmlPage.find_all('table',{'id': 'linups'})[-2]
    judges = [judge for judge in table_linups.find_all('tr')]
    return judges

# PEGA OS JOGADORES QUE FORAM LESIONADOS E OU SUSPENSOS, ha duas colunas: time A e time B
def getInjuries_Suspennsios():
    ''' 
        PEGA OS JOGADORES QUE FORAM LESIONADOS E OU SUSPENSOS, ha duas colunas: time A e time B
        Dados nao estao limpos / not cleaned data
    '''
    try:
        injuries_suspensions_table = htmlPage.find('table',{'id':'team-injuries_suspended'})

        inj_susp_A = injuries_suspensions_table.find_all('td',{'width':'50%'})[0].find_all('tr')
        inj_susp_B = injuries_suspensions_table.find_all('td',{'width':'50%'})[1].find_all('tr')
    except Exception as error:
        if 'AttributeError' in str(type(error)):
            inj_susp_A = np.nan
            inj_susp_B = np.nan
    return inj_susp_A, inj_susp_B
    
# SUBSTITUICOES
def getSubstituitions():
    ''' 
        Pega as susbstituicoes
        Dados nao estao limpos / not cleaned data
    '''
    substituitions_table = htmlPage.find('table',{'id':'team-sub-lineups'})

    substituitions_A = substituitions_table.find_all('td',{'width':'50%'})[0].find_all('tr')
    substituitions_B = substituitions_table.find_all('td',{'width':'50%'})[1].find_all('tr')
    return substituitions_A, substituitions_B

# ESCALACAO
def getAlignment():
    ''' 
        Pega as susbstituicoes
        Dados nao estao limpos / not cleaned data
    '''
    alignment_table = htmlPage.find('table',{'id': 'team-lineups'})

    alignment_A = alignment_table.find_all('td',{'width':'50%'})[0].find_all('tr')
    alignment_B = alignment_table.find_all('td',{'width':'50%'})[1].find_all('tr')
    return alignment_A, alignment_B

# ESQUEMA TATICO / FORMACAO
def getTeamFormation():
    '''
        ESQUEMA TATICO (x-x-x)
    '''
    try:
        team_A_formation = htmlPage.find('div',{'class':'team_A_formation'}).text
        team_B_formation = htmlPage.find('div',{'class':'team_B_formation'}).text
    except Exception as error:
        if 'AttributeError' in str(type(error)):
            team_A_formation = np.nan
            team_B_formation = np.nan
            
    return team_A_formation, team_B_formation

# RESULTADOS DO JOGO
def getResults():
    '''
        Resultado no primeiro tempo e resultado no segundo tempo e resultado do jogo
    '''
    first_time_score = htmlPage.find('td',{'class':'ht-score match-sum-wd-score match-sum-score-1'}).text
    second_time_score = htmlPage.find('td',{'class':'ht2-score match-sum-wd-score match-sum-score-1'}).text.replace(' ','').replace('\n', '')
    full_time_score = htmlPage.find('span',{'id':'match-summary-full-result'}).text.replace(' ','').replace('\n', '')
    
    return {'first_time': first_time_score, 'second_time': second_time_score, 'full_time': full_time_score}


# RESUMO DO JOGO
def getSummary():
    '''
        Salva o resumo do jogo em uma lista
    '''
    
    first_time_table = htmlPage.find('table',{'id':'first-half-summary'}) # RESUMO DO JOGO (1 tempo)
    second_time_table = htmlPage.find('table',{'id':'second-half-summary'}) # RESUMO DO JOGO (2 tempo)

    
    full_time = [first_time_table, second_time_table]
    summary = []
    for game_part in full_time:
        for event in game_part.find_all('tr'):
            try: #algumas linhas podem estar vazias
                events = {'team': '', 'minute': '', 'event': '', 'description': ''}

                # TEAM A EVENTS
                if event.find_all('td',{'class': 'match-sum-wd-minute'})[0].text == '\xa0':
                    pass
                else:
                    events['team'] = 'A'
                    events['minute'] = event.find_all('td',{'class': 'match-sum-wd-minute'})[0].text.replace(' ','').replace('\n', '')
                    events['event'] = event.find_all('td',{'class': 'match-sum-wd-symbol'})[0].find('span').get('title')
                    events['description'] = event.find_all('td',{'class': 'match-sum-wd-description'})[0].text.replace(' ','').replace('\n', '').replace('\xa0', ' ').replace('\r', '')

                # TEAM B EVENTS
                if event.find_all('td',{'class': 'match-sum-wd-minute'})[1].text == '\xa0':
                    pass
                else:
                    events['team'] = 'B'
                    events['minute'] = event.find_all('td',{'class': 'match-sum-wd-minute'})[1].text.replace(' ','').replace('\n', '')
                    events['event'] = event.find_all('td',{'class': 'match-sum-wd-symbol'})[1].find('span').get('title')
                    events['description'] = event.find_all('td',{'class': 'match-sum-wd-description'})[1].text.replace(' ','').replace('\n', '').replace('\xa0', ' ').replace('\r', '')

                summary.append(events) 
            except Exception as error:
                if 'IndexError' in str(type(error)):
                    pass
                else:
                    print('\nError [getSummary()]\n')
                    print(type(error), error)
                    return False
    return summary, first_time_table, second_time_table

# ALGUNS OUTROS DADOS
def getOtherData():
    '''
        Pega alguns outros dados que poderemos usar futuramente:
        * resultados da rodada passada
        * estatisticas da competicao (apos termino)
        * informacoes do evento, como a rodada etc
    '''
    past_round = htmlPage.find('table',{'class': 'competition-rounds'})
    championship_stats = htmlPage.find('div',{'class': 'boxed stats_resume'})
    event_info = htmlPage.find('div',{'class': 'stats-game-head'})
    return {'past_round': past_round, 'championship_stats': championship_stats, 'event_info': event_info}

In [7]:
# PEGA AS ESTATISTICAS
def getStats():
    '''
        Pega as estatisticas da partida:
        * Passe de Bola, * Gols, * Chutes a gol, * Chutes defendidos, * Chutes fora, *Ataques, * Ataques perigosos
        * Impedimentos, * Faltas, * Tiros livres, * Tiro de meta, * Laterais, * Substituicoes, * Escanteios
    '''
    # ESTATISTICAS DA PARTIDA
    stats_table = htmlPage.find('table',{'class':"match_stats_center"})
    keys = {'possession': None, 'goals': None, 'shots_on_target': None, 'shots_blocked': None, 'shots_off_target': None,
            'attacks': None, 'dangerous_attacks': None, 'offsides': None, 'fouls': None, 'free_kicks': None,
            'goal_kicks': None, 'throw_ins': None, 'substitutions': None, 'corners': None}

    team_A = {}
    team_B = {}
    for key in keys.keys():
        try:
            #chutes defendidos
            data = stats_table.find('tr',{'class': key})
            team_A[str(key)+'_A'] = data.find('td',{'class':'stat_value_number_team_A'}).text.replace(' ','').replace('\n','').replace('%', '')
            team_B[str(key)+'_B'] = data.find('td',{'class':'stat_value_number_team_B'}).text.replace(' ','').replace('\n','').replace('%', '')

        except Exception as error:
            qntd_goal = htmlPage.find('span',{'id':'match-summary-full-result'}).text.replace(' ','').replace('\n','').replace('%', '')
            
#             ''' por alguma razao uma das chaves pode aparecer diferente as vezes '''
            if 'AttributeError' in str(type(error)) and key == 'shots_blocked':
                try:
                    key = 'shots_woodwork'
                    data = stats_table.find('tr',{'class': key})
                    team_A[str(key)+'_A'] = data.find('td',{'class':'stat_value_number_team_A'}).text.replace(' ','').replace('\n','').replace('%', '')
                    team_B[str(key)+'_B'] = data.find('td',{'class':'stat_value_number_team_B'}).text.replace(' ','').replace('\n','').replace('%', '')
                except Exception as error:
                    if 'AttributeError' in str(type(error)):
                        team_A[str(key)+'_A'] = np.nan
                        team_B[str(key)+'_B'] = np.nan
            
#             ''' quando nao houver gols no jogo nao havera a key goals'''
            elif 'AttributeError' in str(type(error)) and key == 'goals' and qntd_goal == '0-0':
                team_A[str(key)+'_A'] = '0'
                team_B[str(key)+'_B'] = '0'
            
            elif 'AttributeError' in str(type(error)):
                team_A[str(key)+'_A'] = np.nan
                team_B[str(key)+'_B'] = np.nan
            else:
                print('{} ERROR[getStats()]: '.format(type(error), error))
                return False
    return team_A, team_B

In [8]:
# INSERE OS DADOS NO DATAFRAME
def insertDataToDF(position):
    '''
        recebe uma posicao de linha e insere os dados no df
    '''
#     position = 0
    
    # insere os dados do estadio
    for key in estadio_data.keys():
        data_game.loc[position, key] = estadio_data[key]

    # insere as estatisticas de jogo para cada um dos times
    for keyA, keyB in zip(stat_time_a, stat_time_b):
        data_game.loc[position, keyA] = stat_time_a[keyA]
        data_game.loc[position, keyB] = stat_time_b[keyB]

    #insere os juizes
    data_game.loc[position, 'judges'] = str(judges)

    # insere suspensoes e lesoes
    data_game.loc[position, 'inj_susp_A'] = str(inj_susp_A)
    data_game.loc[position, 'inj_susp_B'] = str(inj_susp_B)

    # insere as substituicoes
    data_game.loc[position, 'substituitions_A'] = str(substituitions_A)
    data_game.loc[position, 'substituitions_B'] = str(substituitions_B)

    # insere a formacao
    data_game.loc[position, 'alignment_A'] = str(alignment_A)
    data_game.loc[position, 'alignment_B'] = str(alignment_B)

    # insere a formacao
    data_game.loc[position, 'team_A_formation'] = team_A_formation
    data_game.loc[position, 'team_B_formation'] = team_B_formation
    
    #insere em formato de texto os links para os 10 jogos passados
    data_game.loc[position, 'last_games_A'] = str(last_games_A)
    data_game.loc[position, 'last_games_B'] = str(last_games_B)
    
    #insere os ultimos jogos entre as duas equipes
    data_game.loc[position, 'last_games_between'] = str(last_games_between)
    
    
    # insere os resultados
    for result in game_result.keys():
        data_game.loc[position, result] = game_result[result]

    # insere se houve gol antes dos 10 minutos
    for event in summary:
        time_event = int(''.join(filter(str.isdigit, event['minute'])))

        if time_event <= 10 and event['event'] == 'Goal':
            print(event)
            data_game.loc[position, 'goal_before_10'] = True
        else:
            data_game.loc[position, 'goal_before_10'] = False
            break

    #insere os dados ~crus~ do sumario
    data_game.loc[position, 'first_time_table'] = str(first_time_table)
    data_game.loc[position, 'second_time_table'] = str(second_time_table)

    #insere o sumario
    data_game.loc[position, 'summary'] = str(summary)

In [9]:
# PEGA OS 10 JOGOS PASSADOS DE CADA EQUIPE
def getTenLastGames():
    '''
        Pega os 10 jogos passados de cada um dos times
    '''
    table_last_events = htmlPage.find('div',{'id': 'ultimos_resultados'})

    team_a, team_b = table_last_events.find_all('td',{'class':'mobile_single_column'})

    past_games_A, past_games_B = [], []
    for line_a, line_b in zip(team_a.find_all('tr'), team_b.find_all('tr')):
        '''itera nas duas colunas ao mesmo tempo pegando as linhas e os links dentro de cada uma das linhas, para time_A e time_B'''
        if len(past_games_A) == 10:
            break
        elif line_a.find('a') != None and line_b.find('a') != None:
            past_games_A.append(line_a.find_all('a')[2].get('href'))
            past_games_B.append(line_b.find_all('a')[2].get('href'))
            
    return past_games_A, past_games_B

In [10]:
# PEGA OS JOGOS PASSADOS ENTRE ESSAS DUAS EQUIPES
def getLastGames():
    '''
        Pega os ultimos jogos entre as duas equipes
    '''
    table_last_games = htmlPage.find('table',{'class':'stat-cd3'})

    last_games = []
    for line in table_last_games.find_all('tr'):
        link = line.find_all('a')
        if link != [] and link[0].text != 'Mostrar todos os jogos':
            last_games.append(link[2].get('href'))
    
    return last_games

In [12]:
# SALVA OS DADOS BASICOS DO DF REFERENCIA DE CADA JOGO PARA O DF COM OS DADOS COMPLETOS
def insertBasicData(position, content):
    ''' insere os dados basicos de cada jogo no df '''
#     position = 0
    for key in content.keys():
        data_game.loc[position, key] = content[key]

In [18]:
files = os.listdir(r'data')

In [30]:
data

NameError: name 'data' is not defined

In [32]:
data = pd.read_csv('data.csv', dtype=str)

In [34]:
data_all_games_day = pd.read_csv('data_games\\01_03_19\\01_03_19.csv', dtype=str)

In [47]:
'''
    Abre os arquivos csv onde contem os jogos de cada dia.
    Cria uma pasta para cada dia salvar os arquivos html dos jogos daquele dia
'''
files = os.listdir(r'data')
# saved_days = []
# data = pd.DataFrame(dtype=str)

for total_days_saved, file in enumerate(files):
    csv_file = 'data\\' + str(file) 
    df = pd.read_csv(csv_file, dtype=str)

    ''' cria o diretorio para o dia '''
    createDirectory(day_event_name=file)
    
    if day_event not in saved_days:
        print('working in file: {}'.format(file))
#         saved = [] #salva as posicoes que ja foram para aquele dia
#         data_all_games_day = pd.DataFrame(dtype=str)
        
        for row, content in df.T.iteritems():
            time.sleep(3)

            data_game = pd.DataFrame(dtype=str)

            if row not in saved:
                print('link para ao jogo {} \n{}'.format(row, content['event_link']))

                req = requests.get(content['event_link'])
                if req.status_code == 200:
                    first_page = req.content
                    htmlPage = bs4.BeautifulSoup(first_page, features='html.parser')

                    ''' cria um diretorio para aquele evento'''
                    createDirectory(event=row)

                    'salva o html da pagina em um arquivo txt'''
                    saveHtml(row, step=1)

                    ''' pega o link e salva em um arquivo csv os outros links fontes de dados'''
                    summary_link = getDataFont()

                    '''basic data game'''
                    insertBasicData(row, content)
                    
                    ''' 10 jogos passados de cada equipe'''
                    last_games_A, last_games_B = getTenLastGames()

                    ''' ultimos jogos entre as duas equipes '''
                    last_games_between = getLastGames()
                    

                    ''' vai para o link com os dados da partida'''
                    req = requests.get(summary_link)
                    if req.status_code == 200:
                        htmlPage = bs4.BeautifulSoup(req.content, features='html.parser')
                        saveHtml(row, step=2)

                        ''' dados do estadio '''
                        estadio_data = getEstadioData()

                        ''' dados estatisticos da partida para cada um dos times '''
                        stat_time_a, stat_time_b = getStats()

                        ''' Arbitros do jogo // dados nao limpos'''
                        judges = getJuizes()

                        ''' Suspensoes e Lesoes // dados nao limpos''' 
                        inj_susp_A, inj_susp_B = getInjuries_Suspennsios()
                        ''' substituicoes // dados nao limpos'''
                        substituitions_A, substituitions_B = getSubstituitions()
                        ''' escalacao-alinhamento - // dados nao limpos'''
                        alignment_A, alignment_B = getAlignment()

                        ''' esquema tatico / formacao'''
                        team_A_formation, team_B_formation = getTeamFormation()

                        ''' resultado do jogo inteiro e resultado parcial no primeiro e segundo tempo'''
                        game_result = getResults()

                        ''' resumo do jogo/eventos ocorridos'''
                        summary, first_time_table, second_time_table = getSummary()

                        ''' dados extras '''
                        extra_data = getOtherData()

                        ''' insere os dados coletados no dataframe passando como linha, a posicao atual no contador'''
                        insertDataToDF(row)

                        data_game.to_csv(str(directory_name + event_name + '/'+ event_name + '.csv'))

                        '''inclui o df no qual possui todos os jogos do dia'''
                        data_all_games_day = data_all_games_day.append(data_game)
                        #event_name foi criada no escopo global dentro da funcao createDirectory
                        print('[saved] {} , total: {} of {}\n'.format(event_name, row, len(df)))

                        saved.append(row)
                    else:
                        print('problem to get data in second request')
                else:
                    print('problem to get data in first request')
#             break

        day_csv_name = str(directory_name + day_event + '.csv')
        # salva os dados dos jogos do dia no df para aquele dia
        data_all_games_day.to_csv(day_csv_name)
        print('\n\n\n######_________ DAY READY: {} _________######\n\n\n'.format(day_event))
        saved = [] #zera o dia
        
        #adiciona ao df principal os dados dos jogos desse dia
        data = data.append(data_all_games_day)
        data.to_csv('data.csv')
        
        saved_days.append(day_event) # salva o dia que teve os dados salvos
        
        break
        if total_days_saved == 3:
            # ira pegar dados dos jogos para 3 dias como teste
            break

working in file: 01_03_19_with_210_events.csv
link para ao jogo 75 
https://www.academiadasapostasbrasil.com/stats/match/mundo/amistosos/riga/paide/2987861
<class 'FileNotFoundError'> [WinError 3] O sistema não pode encontrar o caminho especificado: './data_games/01_03_19/Caramba/Dinamo_v_Paide/'
<class 'FileNotFoundError'> [Errno 2] No such file or directory: './data_games/01_03_19/Caramba/Dinamo_v_Paide/HTML_STATS_Caramba/Dinamo v Paide.txt'


FileNotFoundError: [Errno 2] No such file or directory: './data_games/01_03_19/Caramba/Dinamo_v_Paide/DATA_FONTS_Caramba/Dinamo v Paide.csv'

In [72]:
# df = pd.read_csv('data.csv')
# df

In [73]:
# data
# data_all_games_day
# data_game

In [ ]:
# se der erro 'NewConnectionError' ou 'MaxRetryError' ou 'ConnectionError' ira salvar o que ja fez e parar o loop

# CRIAR VERIFICACAO DE DF:
# * SE NAO EXISTIR DF PARA O DIA EM QUESTAO, IRA CRIAR, SE EXISTIR, IRA VERIFICAR QUAL O ULTIMO JOGO SALVO E IRA COMECAR A PARTIR DALI